## Is there life after graduate school?

### View dashboard visualization: https://share.streamlit.io/saahithirao/bios-823-blog/hw4.py  

### View code: https://github.com/saahithirao/bios-823-blog/blob/master/hw4.py

The goal of this blog post is to explain the process of creating a dashboard visalization using *streamlit* of data about science and engineering PhDs in the US. Datasets used in the dashboard can be found here: https://ncses.nsf.gov/pubs/nsf19301/data. Analysis were performed using the pandas library and visualizations were created using the matplotlib library.

**Doctorate recipients by gender & race from 2008-2016**  
The first visualization I created displays an interactive data table of doctorate recipents by gender and race from 2008 to 2016. The user can click on a year on the sidebar to display data for that year and can select to data by gender or all genders at the same time. Since there was no dataset that contained all of this information, I opted to combine two different datasets: one that contained data on females and on that contained data on males. I extracted the necessary information to create the visualization, as shown below, and merged the two dataframes. Then I created a widget (sidebar) that allows the user to select a specific year that they want to see data for and/or filter the data by gender. 

In [8]:
import pandas as pd
df = pd.read_excel("https://ncses.nsf.gov/pubs/nsf19301/assets/data/tables/sed17-sr-tab021.xlsx", header=3)
df = df.rename(columns={'Ethnicity, race, and citizenship status':'Race'})
df_female = (
        df.
        drop(df[df['Race'].str.contains('citizen')].index.tolist()).
        drop(df[df['Race'].str.contains('visa')].index.tolist()).
        drop(df[df['Race'].str.contains('Hispanic')].index.tolist()).
        drop(df[df['Race'].str.contains('Ethnicity')].index.tolist()).
        reset_index().
        drop(columns = ['index'])
    )
df_female["Gender"] = "Female"
df_female

,Race,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,Gender
0,All doctorate recipients,22494,23187,22488.0,22700,23527,24366,24816,25354,25256,25495,Female
1,American Indian or Alaska Native,70,78,65.0,79,65,64,55,77,73,62,Female
2,Asian,4795,4741,4662.0,4904,5117,5392,5465,5598,5521,5605,Female
3,Black or African American,1396,1552,1403.0,1396,1481,1554,1503,1660,1662,1756,Female
4,White,12704,13058,12525.0,12572,12941,13264,13324,13523,13623,13413,Female
5,More than one race,298,370,418.0,399,484,497,488,535,605,594,Female
6,Other race or race not reported,222,238,205.0,191,202,204,162,167,193,321,Female
